<a href="https://colab.research.google.com/github/kimdarwin/ule/blob/main/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [1]:
!nvidia-smi

Fri Aug 11 04:19:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
'''
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
'''

'\n!pip install -q -U bitsandbytes\n!pip install -q -U git+https://github.com/huggingface/transformers.git \n!pip install -q -U git+https://github.com/huggingface/peft.git\n!pip install -q -U git+https://github.com/huggingface/accelerate.git\n!pip install -q datasets\n'

## 데이터셋: KoAlpaca v1.1a

In [3]:
from datasets import load_dataset

#data = load_dataset("beomi/KoAlpaca-v1.1a")
data = load_dataset('json',data_files='plugin_data.json')

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 147
    })
})

In [5]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    #lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
    lambda x: {'text': f"### 질문: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [6]:
!nvidia-smi


Fri Aug 11 04:19:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import gc
gc.collect()
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [8]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

In [9]:
data['train'][0]['text']

'### 질문: 새 1마리\n\n### 답변: http://shibe.online/api/birds?count=1<|endoftext|>'

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [10]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [13]:
!nvidia-smi

Fri Aug 11 04:22:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    26W /  70W |   9125MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [14]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=120, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.563600
20,1.927400
30,0.948000
40,0.576400
50,0.401800
60,0.365500
70,0.314800
80,0.245900
90,0.200200
100,0.178000


TrainOutput(global_step=700, training_loss=0.25793133769716536, metrics={'train_runtime': 1883.4171, 'train_samples_per_second': 0.743, 'train_steps_per_second': 0.372, 'total_flos': 2262576040058880.0, 'train_loss': 0.25793133769716536, 'epoch': 9.46})

In [15]:
print("wow")

wow


In [16]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [17]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1529: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[    6,     6,     6,  2438,    29,  1832,  4770,   272,    34,    34,
             6,     6,     6,  4253,    29,   224, 11066,    29,    18,    18]])

In [18]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [19]:
gen(data['train'][0]['text'])
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: ### 질문: 새 1마리

### 답변: http://shibe.online/api/birds?count=1<|endoftext|>

### 답변: 21개##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매##### 답변: 21마리 연매####
### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: http://shibe.online/api/birds?count=31&colon=api### #### 답변: http://shibe.online/api/back?count=31&colon=api#### #### 답변: http://shibe.online/api/short?count=31&colon=api#### #### 답변: http://shibe.online/api/from?count=31&colon=api#### #### 답변: http://shibe.online/api/to?count=31&colon=api#### #### 답변: http://shibe.online/api/who?count=31&colon=api#### #### 답변: http://shibe.


`### 질문: 건강하게 살기 위한 세 가지 방법은?`

`### 답변: 첫 번째 방법은 과로하지 않는 것입니다. 두 번째 방법은 물을 많이 마시고, 신선한 음식을 먹는 것입니다. 마지막 방법은 정기적으로 운동을 하는 것입니다. 이러한 세 가지 방법은 규칙적이고 건강하게 사는 데 도움이 됩니다. 하지만 이러한 방법은 누구나 알고 있습니다. 건강한 삶을 살기 위한 다른 방법은 없을까요? 그것은 당신이 가장 만족하지 않는 어떤 것을 추구하는 것 입니다. 예를 들어, 만약 당신이 너무 많이 먹는다고 생각한다면, 당신은 식사량을 줄이는 것이 도움이 될 것입니다. 이러한 연습은 또한 당신의 기분을 더 좋게 해줍니다. 이렇게 당신의 기분이 아주 좋아질 때 당신은 건강한 삶을 살고 있는 것입니다. 이러한 것이 인생에서 가장 만족감을 주는 방식이며, 또한 건강하게 사는 방법입니다. 당신의 건강과 행복을 유지하는 중요한 단계는 어떤 것이 있습니까? 이러한 것을 알아내기 위해 다음 질문에 대해 생각해 보십시오. 그리고 나서 그것들을 적고 목록을 만들어 보십시오. 이 과정은 무엇을 생각하고`

In [20]:
gen(data['train'][0]['text'])
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: ### 질문: 새 1마리

### 답변: http://shibe.online/api/birds?count=1<|endoftext|>

### 답변: http://shibe.online/api/birds?count=1&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&count=0&### 답변: http://shibe.online/api/birds?count=1&count=0&count=0&count=0&count=0&count=0&count=0
### 질문: 슈카월드가 무엇인가요?

### 답변: http://shibe.online/api/count?count=30&stack=####################################...shibe.online#####################################...#####################################...####################################...#####################################...################################


`### 질문: 슈카월드가 무엇인가요?`

`### 답변: 슈카월드는 유튜브 채널이며, 다양한 경제 분야의 주제를 다루고 있습니다. 경제분야에 관심이 있으신 분들은 한번쯤은 들어보셨을 채널일 것입니다. 슈카월드라는 이름은 슈카님의 국적과도 관련이 있는데요, 스위스라는 국명의 '슈카'와 채널의 이름을 따서 슈카월드라고 지은 것으로 보입니다. 현재는 다양한 분야를 다루고 있으며, 여러가지 이슈나 경제 관련 내용이 주로 올라오지만, 구독자 여러분들이 원하는 콘텐츠가 있다면 댓글로 남겨주셔서 채널에 반영하는 등, 시청자의 취향에 따라 다양한 콘텐츠 업데이트 및 업로드가 이루어지고 있습니다. � 인준: 슈카월드는 현재 구독자수 120만 명 이상(!), 조회수는 6600만 뷰 이상을 기록하고 있습니다. 구독자와 조회수의 차이는 컨텐츠를 얼마나 즐겨보는지와 관계가 있는 것으로 보입니다. 슈카월드는 경제관련 내용 이외에도, 여행, 정치, 문화, 역사, 스포츠, 상식, 정보, 시사 등 다양한 분야에 대한 상식, 정보를 다루고 있는 채널입니다. 따라서, 슈카월드를 구독한 많은 분들은`


In [28]:
gen('새 2마리')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 새 2마리

### 답변: http://shibe.online/api/birds?count=2&######################################################################### 답변 ### 깃발 ### 답변: http://shibe.online/api/bonnd?count=2&######################################################################### 30 ## 쏘렌토 35 ##깃 ##...shibe.online › api › find › onDeed?count=30&####...shib


In [29]:
gen('고양이 10마리')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 고양이 10마리

### 답변: http://shibe.online/api/bite/count?count=10&###################################################################################################################################################################################################################################


In [30]:
gen('고양이 1마리')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 고양이 1마리

### 답변: http://shibe.online/api/birds?count=1&#####################################################################################################################################################################################################################################


In [31]:
gen('개 3마리')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 개 3마리

### 답변: http://shibe.online/api/button/on/1241/count?count=3#################################################################둥절########### 악보http://shibe.online/api/button/on/1241/count?count=3###########################################################둥절########## 안내견으리라는 감동 글귀http://shibe.online/api/button/on/1241/count?count=3


In [32]:
gen("새 4마리")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 새 4마리

### 답변: http://shibe.online/api/birds? 강간???????????????????????????????????????????????????????????????????### 답변: http://shibe.online/api/birds????????????????????????????????????????????????????????????????????### 답변: http://shibe.online/api/birds??????????????????????????????????????????손자·손녀의 성추


`### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?`

`### 답변: 마진콜이 발생하는 이유는 은행이 고객에게 지급하고자 하는 금액이 발생한 금액보다 많은 경우, 추가로 내야 하는 금액을 계산해서 계좌에서 더 많은 금액을 인출하게 되는 것입니다. 이렇게 되면, 계좌를 관리하는 은행 입장에서는 계좌에서 추가로 꺼내야 하기 때문에, 은행의 신뢰도가 땅으로 떨어지고, 계좌를 사용하는 은행 고객도 추가로 내야 하는 금액에 대한 불안감을 느낄 수 있습니다. 이런 불안감에 대해 잘 보여주는 사례가 영화 마진콜입니다. 따라서, 이런 현상을 막기 위해서는 고객에게 먼저 지급할 금액과 인출하는 금액을 정확하게 파악하는 것이 중요합니다.`



## 모델 저장 & 업로드

In [ ]:
!huggingface-cli login

In [ ]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')
# 아래 아이디 부분을 수정해서 쓰세요.
#model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')
model.push_to_hub('kkmkorea/qlora-koalpaca-polyglot-12.8b-50step')

- 위 예시코드는 https://huggingface.co/beomi/qlora-koalpaca-polyglot-12.8b-50step 에 올라갑니다. 사용하시는 이름으로 바꿔쓰세요.

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb